# IMPORTS

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import os
import random
import missingno as msno
import pickle
from glob import glob
from sklearn.model_selection import cross_val_score
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import gc
import joblib

from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.1


In [18]:
path = (os.path.abspath("../input"))
sub_path = (os.path.abspath("../submissions"))

sub = pd.read_csv(path + "/sample_submission.csv")
X_train = pd.read_csv(path + '/X_train_lgbm.csv')
X_test = pd.read_csv(path + '/X_test_lgbm.csv')
y_train = pd.read_csv(path + '/y_train_lgbm.csv')

# LGBM

#### -> OOF

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

model = LGBMClassifier(random_state = 44)
lgbm_pred = np.zeros((X_test.shape[0]))
auc_list = []
for tr_idx, val_idx in kf.split(X_train, y_train):
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict_proba(val_x)[:, 1]

    auc = roc_auc_score(val_y, pred)
    auc_list.append(auc)
    
    sub_pred = np.array(model.predict_proba(X_test)[:, 1]) / 5
    lgbm_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 AUC는 {np.mean(auc_list)}')
#1 baseline : 0.8526123651224111
#2 baseline + label encoding : 0.8535428309657258
#3 baseline + grouping : 0.8408434417734512
#4 baseline + binaryohe : 0.8571192393124303
#5 baseline + hourmean + title + 주말주중 : 0.8588049959428016 
#6 baseline + hourmean + title + weekdaylabelencoding: 0.8596225576452744
#6 baseline + hourmean + title + weekdaylabelencoding + cpi_mean + 성실도 + 성실도/status: 0.8605211060959981
#7 baseline + hourmean + title + weekdaylabelencoding + cpi_mean : 0.86045 (최고점)

In [ ]:
sub["STATUS"] = lgbm_pred
sub.to_csv(sub_path + "/lgbm_oof.csv", index = False)

# DNN

In [ ]:
X_train = pd.read_csv(path + '/X_train_dnn.csv')
X_test = pd.read_csv(path + '/X_test_dnn.csv')

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.
def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  

    np.random.seed(1)
    random.seed(2)
    tf.compat.v1.set_random_seed(3)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET") 
   
reset_seeds()

RANDOM SEEDS RESET


In [ ]:
# 학습데이터를 Train/Validation 나누고 학습을 진행함.
i = int(round(X_train.shape[0] * 0.6,0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

In [ ]:
input = keras.Input(shape=(X_train.shape[1],))
with tf.device('/device:GPU:0'):

    x = keras.layers.Dense(16, activation='relu')(input)
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(input, output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', keras.metrics.AUC()])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=50),]
 
    hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                 batch_size=2048, epochs=100, callbacks=callbacks, shuffle=False, verbose=2)

Epoch 1/100
1437/1437 - 5s - loss: 0.5056 - acc: 0.7532 - auc: 0.8063 - val_loss: 0.4859 - val_acc: 0.7539 - val_auc: 0.8316 - 5s/epoch - 4ms/step
Epoch 2/100
1437/1437 - 4s - loss: 0.4865 - acc: 0.7541 - auc: 0.8234 - val_loss: 0.4779 - val_acc: 0.7512 - val_auc: 0.8338 - 4s/epoch - 2ms/step
Epoch 3/100
1437/1437 - 4s - loss: 0.4855 - acc: 0.7511 - auc: 0.8224 - val_loss: 0.4744 - val_acc: 0.7500 - val_auc: 0.8352 - 4s/epoch - 2ms/step
Epoch 4/100
1437/1437 - 4s - loss: 0.4853 - acc: 0.7491 - auc: 0.8217 - val_loss: 0.4725 - val_acc: 0.7502 - val_auc: 0.8359 - 4s/epoch - 2ms/step
Epoch 5/100
1437/1437 - 4s - loss: 0.4851 - acc: 0.7478 - auc: 0.8213 - val_loss: 0.4712 - val_acc: 0.7511 - val_auc: 0.8363 - 4s/epoch - 2ms/step
Epoch 6/100
1437/1437 - 4s - loss: 0.4850 - acc: 0.7468 - auc: 0.8210 - val_loss: 0.4702 - val_acc: 0.7522 - val_auc: 0.8366 - 4s/epoch - 2ms/step
Epoch 7/100
1437/1437 - 4s - loss: 0.4849 - acc: 0.7460 - auc: 0.8208 - val_loss: 0.4693 - val_acc: 0.7533 - val_auc: 

In [ ]:
dnn_pred=model.predict(X_test).flatten()
sub["STATUS"] = dnn_pred
sub.to_csv(sub_path + "/dnn_submission_12071412.csv", index=False)

# ENSENBLE

In [12]:
lgbm = pd.read_csv(sub_path + "/lgbm_oof.csv")
dnn = pd.read_csv(sub_path + "/dnn_submission_12071412.csv")

In [24]:
sub["STATUS"] = (lgbm.STATUS + dnn.STATUS)/2
sub.to_csv(sub_path + "lgbm_dnn.csv", index = False)

,ID,STATUS
0,0,0.905859
1,1,0.064309
2,2,0.541115
3,3,0.877863
4,4,0.069463
...,...,...
1355512,1355512,0.260059
1355513,1355513,0.143008
1355514,1355514,0.151053
1355515,1355515,0.223804
